# MA394 Problem Sheet 2

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random

## Task 1 - Insertion Sort (test insert too)

To begin I shall provide an implementation of shell sort, both because I coded it up last week and would hate for it to go to waste and because (of course) insertion sort is a specific case of shell sort. For this implementation we use the suggested sequence $$Q = \lbrace \left(3^i-1\right)/2 : i=i_{max}, i_{max}-1, \dots, 2, 1 \rbrace .$$
To do insert sort _if you must_ we provide an optional parameter.

In [330]:
# This function takes as an input an (unsorted) *NUMPY ARRAY*, l, of real 
# numbers and outputs a sorted one. This can sort lists of order 2^16 in 
# roughly 10s. To do boring old insert sort set the optional parameter
# insert = "yes". This is VERY slow. It will sort 2^12 in a comparable time
# as shell does 2^16

def shellsort(l,insert = "no"):

    n = len(l) # The length of the list
    
    if insert == "yes":
        Qs = np.array([1])
    else:
        # First we calculate i_max
        iimax = np.floor(np.log(n-1)/np.log(3)).astype(int)
        # Then we generate the linear sequence from i_max to 1
        iis = np.linspace(1,iimax,iimax)
        iis = np.flip(iis).astype(int)
        # Then we generate (and print) the sequence of Q values that we will use
        Qs = (0.5*(np.power(3,iis)-1)).astype(int)
        print("Q values", Qs) 

    # Now we execute the algorithm 
    for q in Qs: # Going through the levels of refinery
        for m in range(q): # And the respective subsequences
            for k in range(np.floor(n/q).astype(int)): # The indicies of 
                # the subsequence in question
                for j in range(k): # To compare to the already sorted
                    if l[q*(k-j)-m]>=l[q*(k-j-1)-m]:
                        break # break if we are greater than OR equal to the
                        # next sorted element
                    else:
                        l[[q*(k-j)-m, q*(k-j-1)-m]]=l[[q*(k-j-1)-m, q*(k-j)-m]]
                        # otherwise switch round the two elements and continue
                        # to compare

                    
                    
    return(l)

### Now to test this algorithm on some lists

To begin, a random one that takes real values both positive and negative.

In [336]:
l = np.random.rand(np.power(2,12),1)-0.5
shellsort(l)

Q values [1093  364  121   40   13    4    1]


array([[-0.49995134],
       [-0.49985456],
       [-0.49958478],
       ...,
       [ 0.49915736],
       [ 0.49963728],
       [ 0.49998682]])

Next, some "tricky" scenarios

In [4]:
# All zeros
l = np.zeros((10,1))
shellsort(l)

Q values [4 1]


array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [5]:
# Already in order
l = np.linspace(0,12,13)
shellsort(l)

Q values [4 1]


array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.])

In [6]:
# Worst case scenario
l = np.linspace(0,12,13)
l = np.flip(l)
shellsort(l)

Q values [4 1]


array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.])

## Task 2 - Mergesort

This feels like plagiarism, but equally it is what you've asked us to do so- voila, your mergesort algorithm (I'll comment it just in case). This runs on the pyton list thingys as opposed to the numpy arrays.

In [7]:
def interlace(list1, list2):
    alist = []
    # base cases
    if (len(list1) == 0): 
        return list2
    elif (len(list2) == 0):
        return list1
    # recursive steps, add the lower elt to alist
    elif list1[0] < list2[0]:
        alist.append(list1[0]) 
        return alist + interlace(list1[1:], list2)
    else:
        alist.append(list2[0])
        return alist + interlace(list1, list2[1:])

In [8]:
def mergeSort(A):
    n=len(A)
    if n == 1:
        return A # an array of length 1 is already sorted
    else:
        m=n//2
    return interlace(mergeSort(A[0:m]), mergeSort(A[m:n]))

### Now to test it

Simple random case

In [9]:
A = random.sample(range(1,200),10)
mergeSort(A)

[8, 20, 32, 50, 51, 55, 133, 135, 172, 195]

Now with replacement and negative numbers 

In [10]:
A = random.choices(range(-5,5), weights=None, cum_weights=None, k=20)
mergeSort(A)

[-5, -5, -4, -4, -4, -3, -2, -2, -2, -2, -2, -1, -1, -1, -1, -1, 0, 2, 4, 4]

Big random case

In [11]:
A = random.sample(range(1,np.power(2,14)),np.power(2,13))
mergeSort(A)

RecursionError: maximum recursion depth exceeded while calling a Python object

Uh oh, maximum recursion depth exceeded, that won't do.

## Task 3 - Fixing Mergesort (test on non int)

The first step is going to be to adapt the code to run on numpy arrays since apparently (source: Oscar) these are faster than python lists. 

In [327]:
# This takes as inputs sorted numpy arrays and outputs their merged array. The
# arrays are to consist of real numbers which may be represented by integers or
# floats

def merge(l1,l2): # The new and improved interlace function
    
    # For ease of implementation we want l2 to contain our largest element
    if np.max(l1)>np.max(l2):
        l1, l2 = l2, l1

    # We initialise alist for speed
    alist = np.zeros(len(l1)+len(l2))
    # We will use a counter to keep track of what index of alist to assign to
    C = 0
    # It will be easier if we can rewrite the already assigned elements of l1
    # to be larger than the biggest element in l2 so they do not get reassigned
    big = np.max(l2) + 1

    for k in l2:
        # We introduce a new temporary counter for the number of elements of l1
        # less than or equal to the current element of l2, this is where 
        # rewriting l1 using'big' helps
        c = np.sum(l1<=k)
        # if c>0 we assign those elements to alist
        if c>0:
            alist[C:C+c] = l1[l1<=k]
            l1[l1<=k] = big
        # We update C
        C = C + c
        # We add the element of l2
        alist[C] = k
        # And update C again
        C = C + 1

    return alist

In [328]:
# Takes a numpy array of real numbers (as integers or floats) and sorts them. 
# It can sort a list of 2^17 elements in around 15s

def mergesort(l): # The new and improved merge sort (notice lowercase s)
    n=len(l)
    if n == 1:
        return l # an array of length 1 is already sorted
    else:
        m=n//2
    return merge(mergesort(l[0:m]), mergesort(l[m:n]))

In [337]:
l = np.random.randint(np.power(2,18),size = (np.power(2,17)))
# print(np.sort(l))
mergesort(l)



array([0.00000e+00, 1.00000e+00, 4.00000e+00, ..., 2.62137e+05,
       2.62138e+05, 2.62140e+05])

## Task 4 - Runtime

In [ ]:
for pows in range(20):
    